In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

!pip install -q wandb
import wandb

     |████████████████████████████████| 2.1MB 6.2MB/s 
     |████████████████████████████████| 163kB 25.4MB/s 
     |████████████████████████████████| 133kB 28.4MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 


In [ ]:
# train_srp53 = pd.read_csv('/content/drive/MyDrive/2040_final_project/Molecular Exploration/Data/sr-p53.smiles',
#                           sep='\t',
#                           names=['smiles', 'id', 'target'])

train_srp53 = pd.read_csv('/content/drive/MyDrive/2040_final_project/Molecular Exploration/Data/SR-p53_train.txt',
                          sep='\s+',
                          names=['smiles', 'target'])

whole_train_srp53 = pd.read_csv('/content/drive/MyDrive/2040_final_project/Molecular Exploration/Data/SR-p53_wholetraining.txt',
                          sep='\s+',
                          names=['smiles', 'target'])

test_srp53 = pd.read_csv('/content/drive/MyDrive/2040_final_project/Molecular Exploration/Data/SR-p53_test.txt',
                          sep='\s+',
                          names=['smiles', 'target'])


# train_srp53 = pd.read_csv('/content/drive/MyDrive/Molecular Exploration/Data/SR-p53_train.txt',
#                           sep='\s',
#                           names=['smiles', 'target'])

# whole_train_srp53 = pd.read_csv('/content/drive/MyDrive/Molecular Exploration/Data/SR-p53_wholetraining.txt',
#                           sep='\s',
#                           names=['smiles', 'target'])

# test_srp53 = pd.read_csv('/content/drive/MyDrive/Molecular Exploration/Data/SR-p53_test.txt',
#                           sep='\s',
#                           names=['smiles', 'target'])

In [ ]:
whole_train_srp53.head()

,smiles,target
0,Br.Br.CN(C)CCCc1cc(C(=O)Nc2ccc(-c3ccncc3)cc2)c...,0
1,Br.Br.N=C(N)SCCN,0
2,Br.C/N=c1/nc(-c2ccccc2)n(-c2ccccc2)s1,0
3,Br.C=CCN1CCc2c(cc(O)c(O)c2Br)[C@@H](c2ccccc2)C1,0
4,Br.C=CCN1CCc2c(cc(O)c(O)c2Cl)C(c2ccccc2)C1,0


In [ ]:
train_srp53.head()

,smiles,target
0,CCN1C(=CC=Cc2sc3ccccc3[n+]2CC)Sc2ccccc21.[I-],1
1,COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@@H]1C(OC)...,1
2,Cc1cccc(C)c1OCC(C)N.Cl,0
3,CO.COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC...,1
4,CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)...,0


In [ ]:
test_srp53.head()

,smiles,target
0,CNc1ncnc2c1ncn2[C@@H]1O[C@H](CO)C(O)[C@H]1O,0
1,Oc1cc(O)cc(/C=C/c2ccc(O)c(O)c2)c1,1
2,COc1ccc2c(c[n+](C)c3c4cc5c(cc4ccc23)OCO5)c1OC....,0
3,Br.Cc1onc(O)c1CC(N)C(=O)O,0
4,Nc1nc(N)c(N=O)c(OCC2CCCCC2)n1,0


In [ ]:
whole_train_srp53['smiles'].iloc[0]

'Br.Br.CN(C)CCCc1cc(C(=O)Nc2ccc(-c3ccncc3)cc2)ccc1O'

In [ ]:
!pip install SmilesPE

In [ ]:
from SmilesPE.pretokenizer import atomwise_tokenizer

smi = 'CC[N+](C)(C)Cc1ccccc1Br'
toks = atomwise_tokenizer(smi)
print(toks)

['C', 'C', '[N+]', '(', 'C', ')', '(', 'C', ')', 'C', 'c', '1', 'c', 'c', 'c', 'c', 'c', '1', 'Br']


***Tokenize all of the training strings***

*Tokenize the training set*

In [ ]:
# traverse through the training smiles and tokenize, and accumulate the alphabet
tkn_smls_train = []
train_alphabet = set()
for sml in train_srp53.smiles:
    toks = atomwise_tokenizer(sml)
    train_alphabet = train_alphabet.union(set(toks))
    tkn_smls_train.append(toks)

In [ ]:
tkn_smls_test = []
for sml in test_srp53.smiles:
    toks = atomwise_tokenizer(sml)
    tkn_smls_test.append(toks)

In [ ]:
print(len(tkn_smls_train))
print(len(train_alphabet))
print(len(tkn_smls_test))

8629
127
268


***Preprocessing sequence data***

*Trying to deal with unseen tokens in test set*

In [ ]:
# copied from https://stackoverflow.com/questions/21057621/sklearn-labelencoder-with-never-seen-before-values
from sklearn.preprocessing import LabelEncoder
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

## EXAMPLE OF UNSEEN TOKEN PROCESSING
# country_list = ['Argentina', 'Australia', 'Canada', 'France', 'Italy', 'Spain', 'US', 'Canada', 'Argentina, ''US']

# label_encoder = LabelEncoderExt()

# label_encoder.fit(country_list)
# print(label_encoder.classes_) # you can see new class called Unknown
# print(label_encoder.transform(country_list))


# new_country_list = ['Canada', 'France', 'Italy', 'Spain', 'US', 'India', 'Pakistan', 'South Africa']
# print(label_encoder.transform(new_country_list))

In [ ]:
# using above object LabelEncoderExt
le = LabelEncoderExt()

In [ ]:
# fit a label encoder on the alphabet from our training data
alphabet_enc = le.fit(list(train_alphabet))

In [ ]:
alphabet_enc.transform(tkn_smls_train[0])

array([ 18,  18,  22,   6,  18,   1,  15,  18,  18,  15,  18, 124,   7,
       127, 124,   8, 124, 124, 124, 124, 124,   8, 116,   7,  18,  18,
         2,  25, 124,   7, 124, 124, 124, 124, 124,   7,   6,   4,  68])

In [ ]:
# traverse through the tokenized training samples and store a numpy array of integer-encoded samples
train_encoded = [alphabet_enc.transform(sm) for sm in tkn_smls_train]

In [ ]:
train_encoded[0]

array([ 18,  18,  22,   6,  18,   1,  15,  18,  18,  15,  18, 124,   7,
       127, 124,   8, 124, 124, 124, 124, 124,   8, 116,   7,  18,  18,
         2,  25, 124,   7, 124, 124, 124, 124, 124,   7,   6,   4,  68])

In [ ]:
test_encoded = [alphabet_enc.transform(sm) for sm in tkn_smls_test]
test_encoded[0]

array([ 18,  22, 124,   6, 125, 124, 125, 124,   7, 124,   6, 125, 124,
       125,   7,  40,   6,  23,  42,   1,  18,  23,   2,  18,   1,  23,
         2,  42,   6,  23])

In [ ]:
import tensorflow as tf

padded_train = tf.keras.preprocessing.sequence.pad_sequences(
    train_encoded, padding="post"
)
padded_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_encoded, padding="post", maxlen=padded_train.shape[1]
)

In [ ]:
padded_train

array([[ 18,  18,  22, ...,   0,   0,   0],
       [ 18,  23,  18, ...,   0,   0,   0],
       [ 18, 124,   6, ...,   0,   0,   0],
       ...,
       [ 25,  15,  18, ...,   0,   0,   0],
       [ 18,  18,  23, ...,   0,   0,   0],
       [ 18,  18,  18, ...,   0,   0,   0]], dtype=int32)

In [ ]:
print(padded_train.shape)
print(padded_test.shape)

(8629, 240)
(268, 240)


***Training data***

*Going to split this for the baseline -- the test file from the NIH doesn't have a target variable*

In [ ]:
# X_train = tf.one_hot(padded_inputs, depth=len(train_alphabet))
X_train = padded_train
X_test = padded_test

y_train = train_srp53.target
y_test = test_srp53.target

In [ ]:
X_train

array([[ 18,  18,  22, ...,   0,   0,   0],
       [ 18,  23,  18, ...,   0,   0,   0],
       [ 18, 124,   6, ...,   0,   0,   0],
       ...,
       [ 25,  15,  18, ...,   0,   0,   0],
       [ 18,  18,  23, ...,   0,   0,   0],
       [ 18,  18,  18, ...,   0,   0,   0]], dtype=int32)

In [ ]:
X_train.shape

(8629, 240)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8629, 240)
(268, 240)
(8629,)
(268,)


### Baseline model

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding

# initialize weights and biases project
wandb.init(project="SMILES_baseline")

wandb: Currently logged in as: cjrohlicek (use `wandb login --relogin` to force relogin)


In [ ]:
embedding_vector_length = len(train_alphabet) + 1
model = Sequential()
model.add(tf.keras.layers.Masking(mask_value=0., input_shape=(X_train.shape[1],)))
model.add(Embedding((len(train_alphabet)+1), embedding_vector_length, input_length=240))
model.add(SimpleRNN(100, input_shape=(X_train.shape[0],X_train.shape[1])))
# model.add(LSTM(100, input_shape=(X_train.shape[0],X_train.shape[1])))
model.add(Dense(1, activation='sigmoid'))

# common metric for the binary classification problem: AUC
auc_metric = tf.keras.metrics.AUC()

opt = keras.optimizers.SGD(lr = 0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy',auc_metric])

print(model.summary())

# need the wandb callback
callbacks = [wandb.keras.WandbCallback()]

model.fit(X_train, 
          y_train, 
          validation_data=(X_test, y_test), 
          epochs=10, 
          batch_size=32,
          callbacks = callbacks)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 240)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 240, 128)          16384     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 100)               22900     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 39,385
Trainable params: 39,385
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
270/270 [==============================] - 25s 88ms/step - loss: 0.5979 - accuracy: 0.9377 - auc_2: 0.5060 - val_loss: 0.4960 - val_accuracy: 0.8955 - val_auc_2: 0.5000
Epoch 2/10
270/270 [==============================] - 23s 87ms/

In [ ]:
wandb.finish()

epoch,9
loss,0.24314
accuracy,0.93777
auc_2,0.5046
val_loss,0.33496
val_accuracy,0.89552
val_auc_2,0.5
_runtime,250
_timestamp,1617659086
_step,9
best_val_loss,0.33482


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▂▂▂▁▁▁▁
accuracy,▁▁▁▁▁▁▁▁▁▁
auc_2,▂█▄▁█▅▇█▁▆
val_loss,█▅▃▂▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_auc_2,▁▁▁▁▁▁▁▁▁▁
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


### ... and the moral is: Because toxicity is less common than non-toxicity, data will be very skewed in favor of non-tox. To combat this and get an informative judge of classification quality, we use AUROC (resilient to skewed data)

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies[0][1]/len(y_train)) #<--- proportion in class 0

0.9377679916560435


In [ ]:
(unique, counts) = np.unique(y_test, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies[0][1]/len(y_test)) #<--- proportion in class 0

0.8955223880597015


-----------------